In [1]:
import tensorflow as tf
from keras.preprocessing import image
import numpy as np
import keras
from keras import Model
import time
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D

fileName = "Trump.jpg"



# Global MobileNets model
mobile = keras.applications.mobilenet.MobileNet(weights="imagenet")
model = Model(mobile.input, mobile.layers[86].output)


/home/nikhil/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def write_to_file_input(fileName, X):
    fhandler = open(fileName, "w")
    for i in range(3):
        for j in range(224):
            for k in range(224):
                fhandler.write(str(X[j][k][i]) + "\n")
    fhandler.close()

In [ ]:
def create_Input_Files():
    
    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalizing the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Write Normalized Input to file
    write_to_file_input("data/FirstLayer/inputNorm.txt", X)
    
    
    # Write combination input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 10
    X[:, :, 2] = 100
    write_to_file_input("data/FirstLayer/inputComb.txt", X)
    
    # Write all 1's input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 1
    X[:, :, 2] = 1
    write_to_file_input("data/FirstLayer/inputSet1.txt", X)

    print("Creating Input files Complete")


In [ ]:
create_Input_Files()

In [ ]:
def write_to_file_weights_layer_one(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(32):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()
    

In [ ]:
W = model.layers[2].get_weights()[0]
write_to_file_weights_layer_one("data/FirstLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_one("data/FirstLayer/weightsSet1.txt", W)

In [ ]:
def write_to_file_weights_layer_second(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(32):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()


In [ ]:
W = model.layers[5].get_weights()[0]
write_to_file_weights_layer_second("data/SecondLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_second("data/SecondLayer/weightsSet1.txt", W)

In [2]:
def checkCalculations():

    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalized the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Set as 1
    #X[:,:,0] = 1
    #X[:,:,1] = 1
    #X[:,:,2] = 1

    # Set as Combination
    #X[:,:,0] = 1
    #X[:,:,1] = 10
    #X[:,:,2] = 100

    print(X[0][0][0])
    print(X[0][1][0])
    print(X[0][2][0])
    X = np.expand_dims(X, axis=0)
    print(X.shape)

    """
    # Weights part
    W = model.layers[2].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[2].set_weights(W)
    
    
    W = model.layers[5].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[5].set_weights(W)
    
    W = model.layers[8].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[8].set_weights(W)
    
    W = model.layers[12].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[12].set_weights(W)
    
    W = model.layers[15].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[15].set_weights(W)
    
    W = model.layers[18].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[18].set_weights(W)
    
    W = model.layers[21].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[21].set_weights(W)
    
    W = model.layers[25].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[25].set_weights(W)
    
    W = model.layers[28].get_weights()[0]
    # Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[28].set_weights(W)
    W = model.layers[28].get_weights()[0]
    
    W = model.layers[31].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[31].set_weights(W)
    W = model.layers[31].get_weights()[0]
    
    W = model.layers[34].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[34].set_weights(W)
    W = model.layers[34].get_weights()[0]
    
    W = model.layers[38].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[38].set_weights(W)
    W = model.layers[38].get_weights()[0]
    
    W = model.layers[41].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[41].set_weights(W)
    W = model.layers[41].get_weights()[0]
    
    W = model.layers[44].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[44].set_weights(W)
    W = model.layers[44].get_weights()[0]
    
    W = model.layers[47].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[47].set_weights(W)
    W = model.layers[47].get_weights()[0]
    
    W = model.layers[50].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[50].set_weights(W)
    W = model.layers[50].get_weights()[0]
    
    W = model.layers[53].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[53].set_weights(W)
    W = model.layers[53].get_weights()[0]
    
    W = model.layers[56].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[56].set_weights(W)
    W = model.layers[56].get_weights()[0]
    
    W = model.layers[59].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[59].set_weights(W)
    W = model.layers[59].get_weights()[0]
    
    W = model.layers[62].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[62].set_weights(W)
    W = model.layers[62].get_weights()[0]
    
    W = model.layers[65].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[65].set_weights(W)
    W = model.layers[65].get_weights()[0]
    
    W = model.layers[68].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[68].set_weights(W)
    W = model.layers[68].get_weights()[0]
    
    W = model.layers[71].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[71].set_weights(W)
    W = model.layers[71].get_weights()[0]
    
    W = model.layers[75].get_weights()[0]
    #Weights set as 1
    W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[75].set_weights(W)
    W = model.layers[75].get_weights()[0]
    """
    
    
    output = model.predict(X)
    print("Output Shape ---> ")
    print(output.shape)
    print("--------------------------------")

    # Save Output to file
    fOutput = open("data/TwentySevenLayer/TwentySeven_Layer_Output.txt", "w")
    for i in range(len(output)):
        for j in range(len(output[0][0][0])):
            for k in range(len(output[0])):
                for l in range(len(output[0][0])):
                    fOutput.write(str(output[i][k][l][j]) + "\n")
    fOutput.close()
    print("Output File Writing Complete!!!")

    print("\n\n Description of Layers")
    for layer in model.layers:
        print(layer)


In [3]:
import time
start = time.time()
print("Total Time -->", (start))
checkCalculations()
end = time.time()
print("Total Time -->", (end - start))


Total Time --> 1550088573.8246968
0.14501739
0.35173547
0.5635475
(1, 224, 224, 3)
Output Shape ---> 
(1, 7, 7, 1024)
--------------------------------
Output File Writing Complete!!!


 Description of Layers
Total Time --> 5.345784664154053


<h2> Checking calculations for Layer 8 </h2>

In [ ]:
def checkFile_EighthLayer():
    fExp = open('data/EighthLayer/Eighth_Layer_Output.txt', 'r')
    fAct = open('data/EighthLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    mismatchStart = 0;
    boo = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            if(boo < 500):
                boo += 1
                print(str(cAct[i].strip()) + "  --->   " + str(cExp[i].strip()) + " <---- " + str(i) )
        
    print("Number of mismatch - " + str(counter))
    
    print(mismatchStart)
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_EighthLayer()

<h2> Checking calculations for Layer 6 </h2>

In [ ]:
def checkFile_SixthLayer():
    fExp = open('data/SixthLayer/Sixth_Layer_Output.txt', 'r')
    fAct = open('data/SixthLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    mismatchStart = 0;
    boo = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            if(boo < 500):
                boo += 1
                print(str(cAct[i].strip()) + "  --->   " + str(cExp[i].strip()) + " <---- " + str(i) )
        
    print("Number of mismatch - " + str(counter))
    
    print(mismatchStart)
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_SixthLayer()

<h2> Checking calculations for Layer 5 </h2>

In [ ]:
def checkFile_FifthLayer():
    fExp = open('data/FifthLayer/Fifth_Layer_Output.txt', 'r')
    fAct = open('data/FifthLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "  --->   " + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_FifthLayer()

<h2> Checking calculations for Layer 1 </h2>

In [ ]:
def checkFile_FirstLayer():
    fExp = open('data/FirstLayer/First_Layer_Output.txt', 'r')
    fAct = open('data/FirstLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_FirstLayer()

<h2> Checking calculations for Layer 4 </h2>

In [ ]:
def checkFile_FourthLayer():
    fExp = open('data/FourthLayer/Fourth_Layer_Output.txt', 'r')
    fAct = open('data/FourthLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "--->" + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_FourthLayer()

<h2> Checking calculations for Layer 2 </h2>

In [ ]:
def checkFile_SecondLayer():
    fExp = open('data/SecondLayer/Second_Layer_Output.txt', 'r')
    fAct = open('data/SecondLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "--->" + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_SecondLayer()

In [ ]:
def checkFile_ThirdLayer():
    fExp = open('data/ThirdLayer/Third_Layer_Output.txt', 'r')
    fAct = open('data/ThirdLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

<h2> Checking calculations for Layer 7 </h2>

In [ ]:
def checkFile_SeventhLayer():
    fExp = open('data/SeventhLayer/Seventh_Layer_Output.txt', 'r')
    fAct = open('data/SeventhLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "--->" + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_SeventhLayer()

<h2> Saving Batch Norm for Layer 6 </h2>

In [ ]:
import math as m
batchNorm = model.layers[19].get_weights()

fMean = open("data/SixthLayer/Sixth_Layer_Mean.txt", "w")
fSD = open("data/SixthLayer/Sixth_Layer_StanDev.txt", "w")
fGamma = open("data/SixthLayer/Sixth_Layer_Gamma.txt", "w")
fBeta = open("data/SixthLayer/Sixth_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Batch Norm for Layer 2 </h2>

In [ ]:
import math as m
batchNorm = model.layers[6].get_weights()

fMean = open("data/SecondLayer/Second_Layer_Mean.txt", "w")
fSD = open("data/SecondLayer/Second_Layer_StanDev.txt", "w")
fGamma = open("data/SecondLayer/Second_Layer_Gamma.txt", "w")
fBeta = open("data/SecondLayer/Second_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Weights for Layer 3  </h2>

In [ ]:
def write_to_file_weights_layer_three(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(1):
            for k in range(64):
                for l in range(32):
                    fTemp.write(str(W[i][j][l][k]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer8 = model.layers[8]
W = np.array(layer8.get_weights())
print(W.shape)

W = layer8.get_weights()[0]
write_to_file_weights_layer_three("data/ThirdLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_three("data/ThirdLayer/weightsSet1.txt", W)

<h2> Saving Batch Norm for Layer 3 </h2>

In [ ]:
import math as m
batchNorm = model.layers[9].get_weights()

fMean = open("data/ThirdLayer/Third_Layer_Mean.txt", "w")
fSD = open("data/ThirdLayer/Third_Layer_StanDev.txt", "w")
fGamma = open("data/ThirdLayer/Third_Layer_Gamma.txt", "w")
fBeta = open("data/ThirdLayer/Third_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Weights for Layer 4  </h2>

In [ ]:
def write_to_file_weights_layer_four(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(64):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer12 = model.layers[12]
W = np.array(layer12.get_weights())
print(W.shape)

W = layer12.get_weights()[0]
write_to_file_weights_layer_four("data/FourthLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_four("data/FourthLayer/weightsSet1.txt", W)

<h2> Saving Batch Norm for Layer 5 </h2>

In [ ]:
import math as m
layer16 = model.layers[16]
W = np.array(layer16.get_weights())
print(W.shape)

batchNorm = model.layers[16].get_weights()

fMean = open("data/FifthLayer/Fifth_Layer_Mean.txt", "w")
fSD = open("data/FifthLayer/Fifth_Layer_StanDev.txt", "w")
fGamma = open("data/FifthLayer/Fifth_Layer_Gamma.txt", "w")
fBeta = open("data/FifthLayer/Fifth_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Batch Norm for Layer 4 </h2>

In [ ]:
import math as m
layer13 = model.layers[13]
W = np.array(layer13.get_weights())
print(W.shape)

fMean = open("data/FourthLayer/Fourth_Layer_Mean.txt", "w")
fSD = open("data/FourthLayer/Fourth_Layer_StanDev.txt", "w")
fGamma = open("data/FourthLayer/Fourth_Layer_Gamma.txt", "w")
fBeta = open("data/FourthLayer/Fourth_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Saving Weights for Layer 5  </h2>

In [ ]:
def write_to_file_weights_layer_five(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(128):
            for k in range(1):
                for l in range(64):
                    fTemp.write(str(W[i][k][l][j]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer15 = model.layers[15]
W = np.array(layer15.get_weights())
print(W.shape)

W = layer15.get_weights()[0]
write_to_file_weights_layer_five("data/FifthLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_five("data/FifthLayer/weightsSet1.txt", W)

<h2> Saving Weights for Layer 6  </h2>

In [ ]:
def write_to_file_weights_layer_six(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(128):
        for j in range(3):
            for k in range(3):
                for l in range(1):
                    fTemp.write(str(W[j][k][i][l]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer18 = model.layers[18]
W = np.array(layer18.get_weights())
print(W.shape)

W = layer18.get_weights()[0]
write_to_file_weights_layer_six("data/SixthLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_six("data/SixthLayer/weightsSet1.txt", W)

<h2> Saving Weights for Layer 7  </h2>

In [ ]:
def write_to_file_weights_layer_seven(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(1):
            for k in range(128):
                for l in range(128):
                    fTemp.write(str(W[i][j][l][k]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer21 = model.layers[21]
W = np.array(layer21.get_weights())
print(W.shape)

W = layer21.get_weights()[0]
write_to_file_weights_layer_seven("data/SeventhLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_seven("data/SeventhLayer/weightsSet1.txt", W)

<h2> Saving Weights for Layer 8  </h2>

In [ ]:
def write_to_file_weights_layer_eight(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(128):
        for j in range(3):
            for k in range(3):
                for l in range(1):
                    fTemp.write(str(W[j][k][i][l]) + "\n")
    fTemp.close()


In [ ]:
for layer in model.layers:
    print(layer)
    
layer25 = model.layers[25]
W = np.array(layer25.get_weights())
print(W.shape)

W = layer25.get_weights()[0]
write_to_file_weights_layer_eight("data/EighthLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_eight("data/EighthLayer/weightsSet1.txt", W)

<h2> Batch Norm for Layer 8 </h2>

In [ ]:
import math as m
layer26 = model.layers[26]
W = np.array(layer26.get_weights())
print(W.shape)

batchNorm = model.layers[26].get_weights()

fMean = open("data/EighthLayer/Eighth_Layer_Mean.txt", "w")
fSD = open("data/EighthLayer/Eighth_Layer_StanDev.txt", "w")
fGamma = open("data/EighthLayer/Eighth_Layer_Gamma.txt", "w")
fBeta = open("data/EighthLayer/Eighth_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Batch Norm for Layer 7 </h2>

In [ ]:
import math as m
layer22 = model.layers[2]
W = np.array(layer22.get_weights())
print(W.shape)

batchNorm = model.layers[22].get_weights()

fMean = open("data/SeventhLayer/Seventh_Layer_Mean.txt", "w")
fSD = open("data/SeventhLayer/Seventh_Layer_StanDev.txt", "w")
fGamma = open("data/SeventhLayer/Seventh_Layer_Gamma.txt", "w")
fBeta = open("data/SeventhLayer/Seventh_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Layer Descriptions </h2>

In [ ]:
for layer in model.layers:
    print(layer)
    
layer25 = model.layers[25]
W = np.array(layer25.get_weights())

print("\n\n Weight Size")
print(W.shape)